 # Data Cleaning with SQL: IMDB Top 1000 Movies

In [1]:
# Importing dependencies for this project.
import sqlite3

## Objective
This notebook is a quick reference example for cleaning data in an SQL database.  
This is a continuation of the [Convert CSV to SQL Database](https://www.kaggle.com/code/oliverdaubney/convert-csv-to-sql-database) using data from kaggle<sup>1</sup>. <br>
Link: __[Original Kaggle Dataset](https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows)__ <br>

## Contents
1. [Cleaning Strategy](#Cleaning-Strategy)
2. [Database Connection](#Database-Connection)
3. [Data Cleaning](#Data-Cleaning)
    1. [Checking for Duplicate Records](#Checking-for-Duplicate-Records)
    2. [Checking for NULL values](#Checking-for-NULL-values)
    3. [Check Attribute Ranges](#Check-Attribute-Ranges)
4. [Validating Data Cleaning](#Validating-Data-Cleaning)
5. [Close Connection to the Database](#Close-Connection-to-the-Database)
6. [References](#References)

## Cleaning Strategy
This is just a simple example data cleaning strategy and it involves three steps.
1. Checking for **Duplicate** records.
2. Checking for **NULL** or empty values.
3. Checking range of values for numerical attributes.

## Database Connection

In [2]:
# Connecting to the database.
connection = sqlite3.connect("data/movies.db")
connection.row_factory = sqlite3.Row
minion = connection.cursor()

In [3]:
# Accessing all the column names.
SQL_command = """
    PRAGMA
        table_info(movies_data);
"""
minion.execute(SQL_command)
column_names = []
output = minion.fetchone()
while output:
    column_names.append(output[1])
    output = minion.fetchone()
column_names

['title',
 'release_year',
 'certificate',
 'runtime',
 'imdb_rating',
 'num_votes',
 'gross']

## Data Cleaning

### Checking for Duplicate Records
The first part of the cleaning involves making sure each record is distinct.

In [4]:
SQL_command = """
    SELECT
        COUNT(title) AS title_count,
        COUNT(DISTINCT title) AS unique_count
    FROM
        movies_data;
"""
minion.execute(SQL_command)
data = minion.fetchone()
print(f'Total Records: {data["title_count"]}')
print(f'Distinct Records: {data["unique_count"]}')

Total Records: 1000
Distinct Records: 999


In [5]:
SQL_command = """
    SELECT
        *
    FROM
        movies_data
    GROUP BY
        title
    HAVING
        COUNT(*) > 1;
"""
minion.execute(SQL_command)
data = minion.fetchone()
while data:
    print(data['title'])
    data = minion.fetchone()

Drishyam


In [6]:
SQL_command = """
    SELECT
        *
    FROM
        movies_data
    WHERE
        title = 'Drishyam';
"""
minion.execute(SQL_command)
data = minion.fetchone()
while data:
    print(f"{data['title']}, {data['release_year']}, {data['runtime']}, {data['imdb_rating']}")
    data = minion.fetchone()

Drishyam, 2013, 160, 8.3
Drishyam, 2015, 163, 8.2


The two movies called 'Drishyam' are different films with the same title, as such, all records are distinct.

### Checking for NULL values
The second part of the cleaning involves checking each field for empty or NULL values.

In [7]:
SQL_command = """
    SELECT
        COUNT(title) AS title_count,
        COUNT(release_year) AS release_year_count,
        COUNT(certificate) AS certificate_count,
        COUNT(runtime) AS runtime_count,
        COUNT(imdb_rating) AS imdb_count,
        COUNT(num_votes) AS num_votes_count,
        COUNT(gross) AS gross_count
    FROM
        movies_data;
"""
minion.execute(SQL_command)
data = minion.fetchone()
for i, key in enumerate(data.keys()):
    print(f'{column_names[i]}: {data[key]}')

title: 1000
release_year: 1000
certificate: 899
runtime: 1000
imdb_rating: 1000
num_votes: 1000
gross: 831


The gross value of the film field is missing for some values, however, we will just restrict the dataset when plotting against this parameter.  
The certificate field is also missing a lot of values and merits further exploration. Let's check what categories are present in the certificate field.

In [8]:
SQL_command = """
    SELECT
        DISTINCT(certificate)
    FROM
        movies_data
"""
minion.execute(SQL_command)
data = minion.fetchone()
while data:
    print(data[0])
    data = minion.fetchone()

A
UA
U
PG-13
R
None
PG
G
Passed
TV-14
16
TV-MA
Unrated
GP
Approved
TV-PG
U/A


Firstly, any NULL certificate can be replaced with the category 'Unknown'.

In [9]:
SQL_command = """
    UPDATE
        movies_data
    SET
        certificate = 'Unknown'
    WHERE
        certificate IS NULL;
"""
minion.execute(SQL_command)
connection.commit()

The certificate field has a lot of categories which are similar e.g PG, UA and TV-PG. These can be combined into a small set of categories (U - Unrestricted, PG - Parental Guidance, R - Restricted).

In [10]:
SQL_command = """
    UPDATE
        movies_data
    SET
        certificate = 'U'
    WHERE
        certificate = 'G';
"""
minion.execute(SQL_command)
connection.commit()

SQL_command = """
    UPDATE
        movies_data
    SET
        certificate = 'PG'
    WHERE
        certificate = 'UA' OR
        certificate = 'PG-13' OR
        certificate = 'Passed' OR
        certificate = 'TV-14' OR
        certificate = '16' OR
        certificate = 'Unrated' OR
        certificate = 'GP' OR
        certificate = 'Approved' OR
        certificate = 'TV-PG' OR
        certificate = 'U/A';
"""
minion.execute(SQL_command)
connection.commit()

SQL_command = """
    UPDATE
        movies_data
    SET
        certificate = 'R'
    WHERE
        certificate = 'A' OR
        certificate = 'TV-MA';
"""
minion.execute(SQL_command)
connection.commit()

### Check Attribute Ranges
The ranges for numerical values can be used to check if they are reasonable or if there are strange outliers that may need cleaning.

In [11]:
SQL_command = """
    SELECT
        MIN(release_year) AS min_release_year,
        MAX(release_year) AS max_release_year,
        MIN(runtime) AS min_runtime,
        MAX(runtime) AS max_runtime,
        MIN(imdb_rating) AS min_imdb_rating,
        MAX(imdb_rating) AS max_imdb_rating,
        MIN(num_votes) AS min_num_votes,
        MAX(num_votes) AS max_num_votes,
        MIN(gross) AS min_gross,
        MAX(gross) AS max_gross
    FROM
        movies_data;
"""
minion.execute(SQL_command)
data = minion.fetchone()
print(f'Release Year - Min:{data["min_release_year"]}, Max:{data["max_release_year"]}')
print(f'Runtime - Min:{data["min_runtime"]}, Max:{data["max_runtime"]}')
print(f'IMDB Rating - Min:{data["min_imdb_rating"]}, Max:{data["max_imdb_rating"]}')
print(f'Number of Votes - Min:{data["min_num_votes"]}, Max:{data["max_num_votes"]}')
print(f'Gross Value - Min:{data["min_gross"]}, Max:{data["max_gross"]}')

Release Year - Min:1920, Max:PG
Runtime - Min:45, Max:321
IMDB Rating - Min:7.6, Max:9.3
Number of Votes - Min:25088, Max:2343110
Gross Value - Min:1,000,045, Max:985,912


There is clearly a data error in the release_year column, so we will find the record and update it.

In [12]:
SQL_command = """
    SELECT
        *
    FROM
        movies_data
    WHERE
        release_year = 'PG';
"""
minion.execute(SQL_command)
data = minion.fetchone()
for i, key in enumerate(data.keys()):
    print(f'{column_names[i]}: {data[key]}')

title: Apollo 13
release_year: PG
certificate: U
runtime: 140
imdb_rating: 7.6
num_votes: 269197
gross: 173,837,933


In [13]:
SQL_command = """
    UPDATE
        movies_data
    SET
        release_year = 1995
    WHERE
        title = 'Apollo 13'
"""
minion.execute(SQL_command)
connection.commit()

The gross value column has a rather large error from when we set up the database. The values are not actually numbers but strings, so we can extract the whole column, edit and update the database.

In [14]:
SQL_command = """
    SELECT
        rowid,
        gross
    FROM
        movies_data;
"""
minion.execute(SQL_command)
values = []
data = minion.fetchone()
while data:
    values.append([data[0], data[1]])
    data = minion.fetchone()

In [15]:
for value in values:
    if value[1] != None:
        value[1] = int(value[1].replace(',', ''))

In [16]:
for value in values:
    if value[1] != None:
        SQL_command = f"""
            UPDATE
                movies_data
            SET
                gross = {value[1]}
            WHERE
                rowid = {value[0]}
        """
        minion.execute(SQL_command)
        connection.commit()

## Validating Data Cleaning
The process of validating the cleaning is very similar to the cleaning. We just want to check that the changes made to the data have actually happened.
1. We have already established that all records are unique during the cleaning.
2. There should be no NULLS in the for certificates.

In [17]:
SQL_command = """
    SELECT
        COUNT(title) AS title_count,
        COUNT(release_year) AS release_year_count,
        COUNT(certificate) AS certificate_count,
        COUNT(runtime) AS runtime_count,
        COUNT(imdb_rating) AS imdb_count,
        COUNT(num_votes) AS num_votes_count,
        COUNT(gross) AS gross_count
    FROM
        movies_data;
"""
minion.execute(SQL_command)
data = minion.fetchone()
for i, key in enumerate(data.keys()):
    print(f'{column_names[i]}: {data[key]}')

title: 1000
release_year: 1000
certificate: 1000
runtime: 1000
imdb_rating: 1000
num_votes: 1000
gross: 831


3. All the numerical data ranges should be reasonable.

In [18]:
SQL_command = """
    SELECT
        MIN(release_year) AS min_release_year,
        MAX(release_year) AS max_release_year,
        MIN(runtime) AS min_runtime,
        MAX(runtime) AS max_runtime,
        MIN(imdb_rating) AS min_imdb_rating,
        MAX(imdb_rating) AS max_imdb_rating,
        MIN(num_votes) AS min_num_votes,
        MAX(num_votes) AS max_num_votes,
        MIN(gross) AS min_gross,
        MAX(gross) AS max_gross
    FROM
        movies_data;
"""
minion.execute(SQL_command)
data = minion.fetchone()
print(f'Release Year - Min:{data["min_release_year"]}, Max:{data["max_release_year"]}')
print(f'Runtime - Min:{data["min_runtime"]}, Max:{data["max_runtime"]}')
print(f'IMDB Rating - Min:{data["min_imdb_rating"]}, Max:{data["max_imdb_rating"]}')
print(f'Number of Votes - Min:{data["min_num_votes"]}, Max:{data["max_num_votes"]}')
print(f'Gross Value - Min:{data["min_gross"]}, Max:{data["max_gross"]}')

Release Year - Min:1920, Max:2020
Runtime - Min:45, Max:321
IMDB Rating - Min:7.6, Max:9.3
Number of Votes - Min:25088, Max:2343110
Gross Value - Min:1305.0, Max:936662225.0


## Close Connection to the Database

In [19]:
connection.close()

## References
1. The dataset was prepared by Harshit Shankhdhar from IMDB available information for their top-rated 1000 movies. Harshit Shankhdhar (2020). <i>IMDB Movies Dataset</i> [Data set]. Kaggle. https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows